In [7]:
import os, json, time, datetime, random, glob, shutil
from pathlib import Path
from typing import Dict, Any
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from cnn_model import run_hyperparameter_search, evaluate_on_test, TimeSeriesDataset, CNNRegressor, train_model, split_LOPO
from torch.utils.data import DataLoader
import neurokit2 as nk  # pip install neurokit2
import seaborn as sns
from ml_dataloader import process_physiology_data, extract_raw_physio_windows
from ml_utils import (
    set_reproducible, dbg, to_NTC_strict, sample_or_load_fixed_test_pids,
    make_participant_disjoint_masks, assert_no_pid_overlap, run_planA_one_mode,
    negative_controls_once, summarize_split_scores, ensure_dir, center_from_train_split,
    write_seed_metrics_csv, count_lstm_params, make_masks_from_fixed_test, recipe_for,
    hv_mask_from_train_x, hv_mask_from_train_y, apply_per_split_mask, assert_no_pid_overlap,
    set_seed, split_across_with_gap, to_loader, plot_train_val_loss
)
from ml_pipeline import (
        run_ablation, select_features_by_ablation, run_grid_search,
        train_and_evaluate_seeds, summarize_test_results
    )

## Dataloader

In [ ]:
extract_raw_physio_windows(
    data_path="./MPA_VISE",
    output_path="./ml_processed_raw",
    window_seconds=5,
    stride_seconds=2,
    sampling_rate=30,
    scenes="Hallway",
    enable_feature_expansion=True,
    fe_diff_orders=(1,2),
    fe_ma_seconds=(2,),
    fe_std_seconds=(5,),
    fe_enable_slope=True,
    fe_enable_iqr=True,
    fe_enable_band_energy=False,
    enable_target_smoothing=True,
    target_smoothing_method="ema",
    target_smoothing_steps=15,   # ≈0.5초 @30Hz
    smooth_before_zscore=True
)


Extracting Raw Signals: 100%|██████████| 108/108 [02:00<00:00,  1.11s/it]


✅ 저장 완료: ./ml_processed_raw
📊 X shape: (11979, 77, 150) | y shape: (11979,) | #PIDs: 103
🧩 Channels: 77 | (예: ['EDA_Tonic' 'EDA_Tonic_diff1' 'EDA_Tonic_diff2' 'EDA_Tonic_ma60'
 'EDA_Tonic_std150' 'EDA_Tonic_slope' 'EDA_Tonic_iqr' 'EDA_Phasic'
 'EDA_Phasic_diff1' 'EDA_Phasic_diff2'])
📝 saved: scene_array.npy, windex_array.npy, feature_tag_list.npy, meta.json


In [3]:
X_array = np.load("./ml_processed_raw/X_array.npy")       # shape: [N, C, T]
y_array = np.load("./ml_processed_raw/y_array.npy")       # shape: [N]
pid_array = np.load("./ml_processed_raw/pid_array.npy")   # shape: [N]

## Window 5 stride 2 30hz_Hallway

In [4]:
# -*- coding: utf-8 -*-
# Leakage-safe main (lag OFF / (N,T,C) / publication-ready)
# - Split FIRST (GAP=10), then derive ALL thresholds/statistics from TRAIN ONLY
# - Train-derived HV mask (default: X-variance), Train-only target centering
# - External-val grid (no internal split), seed ensemble (10) for final report
# - Deterministic settings for reproducibility

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["PYTHONHASHSEED"] = "42"

# Reproducibility / determinism
torch.set_float32_matmul_precision("high")  # set "highest" to disable TF32 if needed
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
try:
    torch.use_deterministic_algorithms(True, warn_only=True)
except Exception:
    pass

seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(seed)

In [ ]:
# ===========================
# 0) User config (only edit this part)
# ===========================
model_type = "LSTM"   # {"CNN","GRU","GRU_Attn","LSTM"} 
DATA_DIR   = "ml_processed_raw"
OUT_DIR    = f"results_planA_fixed_test_{model_type}"

# Split / masking policy
VAL_RATIO   = 0.20
GAP_STEPS   = 10               # Reduce Leakage (in 30Hz, 10 steps = 0.33s)
HV_MODE     = "none"           # {"x_variance","y_train","none"}
HV_QUANTILE = 0.25

# Seeds / Epochs
SEED_MASTER = 42
seed        = SEED_MASTER      # (Important) Fix definition to avoid undefined usage
NUM_SEEDS_FINAL = 20
EPOCHS_FINAL    = 50

# Early stopping
patience_ablation   = 10; min_delta_ablation = 1e-6
patience_grid       = 7 ; min_delta_grid     = 1e-5
patience_train      = 7 ; min_delta_train    = 1e-3

# Grid / Ablation toggle
RUN_ABLATION    = True
ABLATION_EPOCHS = 10
RUN_GRID        = True
GRID_EPOCHS     = 20

# Fine-tune option (for within-participant etc. extension; default pipeline is across)
USE_ACROSS_FINAL_CKPTS = True
FINETUNE_VAL_FRACTION  = 0.20
FINETUNE_EPOCHS        = 8
FINETUNE_PATIENCE      = 3

# Negative controls (Default OFF)
RUN_NEGCTRL = False

# ===========================
# 1) Reproducibility & Device
# ===========================
os.makedirs(OUT_DIR, exist_ok=True)
random.seed(SEED_MASTER)
np.random.seed(SEED_MASTER)
torch.manual_seed(SEED_MASTER)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED_MASTER)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ts_tag = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# ===========================
# 2) Model-specific parameter space switch
# - CNN uses (N,C,T) input internally → check loader conversion
# - RNN types (GRU/GRU_Attn/LSTM) use (N,T,C)
# ===========================
FIXED_DEFAULTS: Dict[str, Dict[str, Any]] = {
    "CNN": {
        "num_filters": 64,
        "kernel_size": 5,
        "dropout": 0.3,
        "batch_size": 32,
        "learning_rate": 1e-3,
         # "input_channels": new_C  # Runtime injection
    },
    "GRU": {
        "hidden_size": 64,
        "num_layers": 1,
        "dropout": 0.3,
        "batch_size": 32,
        "learning_rate": 1e-3,
        # "input_size": C  # Runtime injection
    },
    "GRU_Attn": {
        "hidden_size": 64,
        "num_layers": 1,
        "dropout": 0.3,
        "batch_size": 32,
        "learning_rate": 1e-3,
        # "input_size": C
    },
    "LSTM": {
        "hidden_size": 64,
        "num_layers": 1,
        "dropout": 0.3,
        "batch_size": 32,
        "learning_rate": 1e-3,
        # "input_size": C
    },
}

SEARCH_SPACES: Dict[str, Dict[str, Any]] = {
    "CNN": {
        "num_filters": [16, 32, 64],
        "kernel_size": [3, 5, 7],  # Effective length check will be done at runtime
        "dropout": [0.3, 0.4, 0.5, 0.6],
        "batch_size": [32],
        "learning_rate": [1e-3, 3e-4, 1e-4],
    },
    "GRU": {
        "hidden_size": [32, 64, 128],
        "num_layers": [1, 2],
        "dropout": [0.2, 0.3, 0.4, 0.5, 0.6],
        "batch_size": [32],
        "learning_rate": [1e-3, 3e-4, 1e-4],
    },
    "GRU_Attn": {
        "hidden_size": [32, 64, 128],
        "num_layers": [1, 2],
        "dropout": [0.2, 0.3, 0.4, 0.5, 0.6],
        "batch_size": [32],
        "learning_rate": [1e-3, 3e-4, 1e-4],
    },
    "LSTM": {
        "hidden_size": [32, 64, 128],
        "num_layers": [1, 2],
        "dropout": [0.2, 0.3, 0.4, 0.5, 0.6],
        "batch_size": [32],
        "learning_rate": [1e-3, 3e-4, 1e-4],
    },
}

HEAD_NAMES_PER_MODEL = {
    "CNN":      ["head", "fc", "out", "regressor"],
    "GRU":      ["head", "fc", "out", "regressor"],
    "GRU_Attn": ["head", "fc", "out", "regressor", "proj"],
    "LSTM":     ["head", "fc", "out", "regressor"],
}

assert model_type in FIXED_DEFAULTS, f"Unknown model_type: {model_type}"

# ===========================
# 3) Data load + enforce (N,T,C)
# ===========================
X = np.load(f"{DATA_DIR}/X_array.npy")                 # (N,C,T) or (N,T,C)
y = np.load(f"{DATA_DIR}/y_array.npy").astype(np.float32)
pid = np.load(f"{DATA_DIR}/pid_array.npy")
scene = np.load(f"{DATA_DIR}/scene_array.npy")
widx = np.load(f"{DATA_DIR}/windex_array.npy")
feature_tag_list = np.load(f"{DATA_DIR}/feature_tag_list.npy", allow_pickle=True).tolist()

X = to_NTC_strict(X, feature_tag_list)                # (N,T,C)
N, T, C = X.shape
dbg(f"Normalized to (N,T,C)={X.shape}, |features|={len(feature_tag_list)}")
assert len(feature_tag_list) == C, f"len(feature_tag_list)={len(feature_tag_list)} != C={C}"

# ===========================
# 4) Split (GAP) → HV(train only) → Split∩HV → Train-only Centering
# ===========================
train_m, val_m, test_m, split_info = split_across_with_gap(
    pid, scene, widx, val_ratio=VAL_RATIO, gap_steps=GAP_STEPS, seed=seed
)
dbg(f"SPLIT ready (GAP={GAP_STEPS}).")

if HV_MODE == "none":
    keep_all = np.ones_like(y, dtype=bool)
elif HV_MODE == "x_variance":
    keep_all = hv_mask_from_train_x(X, train_m, q=HV_QUANTILE)
else:  # "y_train"
    keep_all = hv_mask_from_train_y(y, pid, scene, train_m, q=HV_QUANTILE)
dbg(f"HV_MODE={HV_MODE}, keep_ratio={keep_all.mean():.3f}")

(TR, VA, TE) = apply_per_split_mask(X, y, pid, scene, widx, train_m, val_m, test_m, keep_all)
X_train_raw, y_train_raw, pid_train, scene_train, _ = TR
X_val_raw,   y_val_raw,   pid_val,   scene_val,   _ = VA
X_test_raw,  y_test_raw,  pid_test,  scene_test,  _ = TE

assert_no_pid_overlap(pid_train, pid_test)

center_fn, stat = center_from_train_split(y_train_raw, pid_train, scene_train)
y_train = center_fn(y_train_raw, pid_train, scene_train)
y_val   = center_fn(y_val_raw,   pid_val,   scene_val)
y_test  = center_fn(y_test_raw,  pid_test,  scene_test)

X_train, X_val, X_test = X_train_raw, X_val_raw, X_test_raw
dbg(f"Post-centering: train={X_train.shape}, val={X_val.shape}, test={X_test.shape}")

# Save split record
split_json = {
    "val_ratio": VAL_RATIO,
    "gap_steps": GAP_STEPS,
    "seed": seed,
    "HV_MODE": HV_MODE,
    "HV_QUANTILE": HV_QUANTILE,
    "train_pids": sorted(list(set(pid_train.tolist()))),
    "val_pids":   sorted(list(set(pid_val.tolist()))),
    "test_pids":  sorted(list(set(pid_test.tolist()))),
}
with open(os.path.join(OUT_DIR, f"split_{model_type}_{ts_tag}.json"), "w", encoding="utf-8") as f:
    json.dump(split_json, f, ensure_ascii=False, indent=2)

# ===========================
# 5) Ablation (mask-based; leakage-safe)
# ===========================
# base params for ablation/grid
fixed_params = dict(FIXED_DEFAULTS[model_type])

# (CNN only) input_channels injected after feature selection
if RUN_ABLATION:
    # Common ablation call (branch inside by model_type)
    ablation_path = os.path.join(OUT_DIR, f"ablation_{model_type}_{ts_tag}.csv")
    df_ablation = run_ablation(
        X_train, y_train, pid_train,
        X_val,   y_val,   pid_val,
        feature_tag_list,
        model_type=model_type,
        fixed_params=fixed_params,
        seed=seed,
        num_epochs=ABLATION_EPOCHS,
        save_path=ablation_path,
        patience=patience_ablation,
        min_delta=min_delta_ablation
    )
    keep_features, keep_indices = select_features_by_ablation(df_ablation, feature_tag_list, threshold=0.0005)
    assert len(keep_indices) > 0, "No features selected by ablation."
else:
    keep_indices = np.arange(X_train.shape[-1]).tolist()
    keep_features = feature_tag_list

# Channel slice
X_train = X_train[:, :, keep_indices]
X_val   = X_val[:,   :, keep_indices]
X_test  = X_test[:,  :, keep_indices]
feature_tag_list = (np.array(feature_tag_list)[keep_indices]).tolist()
new_C = X_train.shape[-1]
dbg(f"After selection: X_train={X_train.shape}, X_val={X_val.shape}, X_test={X_test.shape}, new_C={new_C}")

# ===========================
# 6) Grid Search (external validation) — model-specific space/injection
# ===========================
search_space = {k: list(v) for k, v in SEARCH_SPACES[model_type].items()}

# Pre-loader check (especially CNN kernel size validity)
if model_type == "CNN":
    # CNN assumes to_loader handles the (N,C,T) transformation internally
    tmp_loader = to_loader(X_train, y_train, model_type="CNN", batch_size=1, shuffle=False, input_channels=new_C)
    xb, _ = next(iter(tmp_loader))
    # Expectation: (1, new_C, T)
    L_eff = xb.shape[-1]
    # Adjust kernel_size validity
    if "kernel_size" in search_space:
        search_space["kernel_size"] = [k for k in search_space["kernel_size"] if k <= L_eff]
    # Inject runtime params
    fixed_params["input_channels"] = new_C
else:
    # RNN types use input_size=C
    fixed_params["input_size"] = new_C

dbg(f"[GRID-PRECHECK] model={model_type} | channels={new_C} | space_keys={list(search_space.keys())}")

if RUN_GRID:
    best_params, _ = run_grid_search(
        X_train, y_train, pid_train,
        model_type=model_type,
        search_space=search_space,
        seed=seed,
        num_epochs=GRID_EPOCHS,
        patience=patience_grid,
        min_delta=min_delta_grid,
        use_internal_split=False,
        external_val_data=(X_val, y_val)
    )
    best_params = dict(best_params)
else:
    # Grid skip → Use defaults
    best_params = dict(FIXED_DEFAULTS[model_type])

# Runtime injection of required params
if model_type == "CNN":
    best_params["input_channels"] = new_C
else:
    best_params["input_size"] = new_C

# Save
grid_best_path = os.path.join(OUT_DIR, f"grid_best_{model_type}_{ts_tag}.json")
with open(grid_best_path, "w", encoding="utf-8") as f:
    json.dump(best_params, f, ensure_ascii=False, indent=2)

dbg(f"[GRID] best_params={best_params}")

# ===========================
# 7) Train on train+val → Test (NUM_SEEDS_FINAL)
# ===========================
X_trainval = np.concatenate([X_train, X_val], axis=0)
y_trainval = np.concatenate([y_train, y_val], axis=0)
pid_trainval = np.concatenate([pid_train, pid_val], axis=0)

# Safety
if model_type == "CNN":
    assert X_trainval.shape[-1] == X_test.shape[-1] == best_params["input_channels"], \
        f"C mismatch: trainval C={X_trainval.shape[-1]}, test C={X_test.shape[-1]}, param={best_params['input_channels']}"
else:
    assert X_trainval.shape[-1] == X_test.shape[-1] == best_params["input_size"], \
        f"C mismatch: trainval C={X_trainval.shape[-1]}, test C={X_test.shape[-1]}, param={best_params['input_size']}"
assert_no_pid_overlap(pid_trainval, pid_test)


train_losses, val_losses, test_scores, train_scores, val_scores  = train_and_evaluate_seeds(
    X_trainval, y_trainval, pid_trainval,
    X_test, y_test,
    model_type=model_type,
    best_params=best_params,
    device=device,
    num_seeds=NUM_SEEDS_FINAL,
    num_epochs=EPOCHS_FINAL,
    patience=patience_train,
    min_delta=min_delta_train
)

# ===========================
# 8) Summaries + META Save
# ===========================
summarize_test_results(test_scores)

meta = {
    "timestamp": ts_tag,
    "model_type": model_type,
    "DATA_DIR": DATA_DIR,
    "OUT_DIR": OUT_DIR,
    "format": "(N,T,C)",
    "channels": int(new_C),
    "GAP_STEPS": GAP_STEPS,
    "HV_MODE": HV_MODE,
    "HV_QUANTILE": HV_QUANTILE,
    "VAL_RATIO": VAL_RATIO,
    "SEED_MASTER": SEED_MASTER,
    "NUM_SEEDS_FINAL": NUM_SEEDS_FINAL,
    "EPOCHS_FINAL": EPOCHS_FINAL,
    "RUN_ABLATION": RUN_ABLATION,
    "ABLATION_EPOCHS": ABLATION_EPOCHS,
    "RUN_GRID": RUN_GRID,
    "GRID_EPOCHS": GRID_EPOCHS,
    "best_params": best_params,
    "center_stat": {k: (float(v) if hasattr(v, "__float__") else v) for k, v in stat.items()},
    "split_info": split_info,
    "HEAD_NAMES": HEAD_NAMES_PER_MODEL.get(model_type, []),
    "lag": "DISABLED",
}
with open(os.path.join(OUT_DIR, f"meta_{model_type}_{ts_tag}.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

# ===========================
# 9) Save artifacts for later plotting
# ===========================
import os, re, json, glob, shutil
import numpy as np
import pandas as pd

os.makedirs(OUT_DIR, exist_ok=True)

# 9-1) Seed-wise scores CSV (train / val / test)
rows = []
for s in range(len(test_scores)):
    tr = train_scores[s] if s < len(train_scores) else (np.nan, np.nan, np.nan)
    va = val_scores[s]   if s < len(val_scores)   else (np.nan, np.nan, np.nan)
    te = test_scores[s]  if s < len(test_scores)  else (np.nan, np.nan, np.nan)
    rows.append({"seed": s, "split": "train", "r2": tr[0], "rmse": tr[1], "mae": tr[2]})
    rows.append({"seed": s, "split": "val",   "r2": va[0], "rmse": va[1], "mae": va[2]})
    rows.append({"seed": s, "split": "test",  "r2": te[0], "rmse": te[1], "mae": te[2]})

df_scores = pd.DataFrame(rows, columns=["seed", "split", "r2", "rmse", "mae"])
scores_csv = os.path.join(OUT_DIR, f"seeds_scores_{model_type}_{ts_tag}.csv")
df_scores.to_csv(scores_csv, index=False, encoding="utf-8")
print(f"[SAVED] {scores_csv}")

# 9-2) Loss curves (list-of-lists) → JSON
loss_json = {
    "train_losses": [[float(x) for x in (lst or [])] for lst in (train_losses or [])],
    "val_losses":   [[float(x) for x in (lst or [])] for lst in (val_losses or [])],
}
loss_path = os.path.join(OUT_DIR, f"loss_curves_{model_type}_{ts_tag}.json")
with open(loss_path, "w", encoding="utf-8") as f:
    json.dump(loss_json, f, ensure_ascii=False, indent=2)
print(f"[SAVED] {loss_path}")

# 9-3) Selected feature tags snapshot (after ablation/selection)
feat_path = os.path.join(OUT_DIR, f"features_selected_{model_type}_{ts_tag}.json")
with open(feat_path, "w", encoding="utf-8") as f:
    json.dump({"features": feature_tag_list, "num_features": int(len(feature_tag_list))}, f, ensure_ascii=False, indent=2)
print(f"[SAVED] {feat_path}")

# 9-4) Prediction files: move into OUT_DIR and index them
# evaluate_and_save()가 CWD에 저장했을 수 있으므로 패턴으로 수집
pred_pat = f"{model_type.lower()}_test_predictions_seed*.npz"
pred_files = glob.glob(pred_pat)
pred_index = []

for src in pred_files:
    # seed 번호 추출
    m = re.search(r"seed(\d+)\.npz$", src)
    seed_str = m.group(1) if m else "NA"
    dst = os.path.join(OUT_DIR, f"{model_type.lower()}_test_predictions_seed{seed_str}_{ts_tag}.npz")
    try:
        shutil.move(src, dst)
    except Exception:
        # 이미 OUT_DIR로 저장된 경우 copy 시도 or skip
        if os.path.abspath(os.path.dirname(src)) != os.path.abspath(OUT_DIR):
            shutil.copy2(src, dst)
    pred_index.append({"seed": int(seed_str) if seed_str.isdigit() else np.nan, "path": dst})

# 이미 OUT_DIR에 저장된 경우(코드가 evaluate_and_save에서 경로를 받도록 구현된 경우)도 다시 스캔
pred_files_out = glob.glob(os.path.join(OUT_DIR, f"{model_type.lower()}_test_predictions_seed*.npz"))
for p in pred_files_out:
    m = re.search(r"seed(\d+)", os.path.basename(p))
    seed_str = m.group(1) if m else "NA"
    if not any(d["path"] == p for d in pred_index):
        pred_index.append({"seed": int(seed_str) if seed_str.isdigit() else np.nan, "path": p})

pred_csv = os.path.join(OUT_DIR, f"predictions_index_{model_type}_{ts_tag}.csv")
pd.DataFrame(pred_index, columns=["seed", "path"]).to_csv(pred_csv, index=False, encoding="utf-8")
print(f"[SAVED] {pred_csv}")

# 9-5) Compact experiment summary for quick plotting dashboards
summary = {
    "timestamp": ts_tag,
    "model_type": model_type,
    "num_seeds": len(test_scores),
    "scores_csv": os.path.basename(scores_csv),
    "loss_curves_json": os.path.basename(loss_path),
    "features_json": os.path.basename(feat_path),
    "predictions_index_csv": os.path.basename(pred_csv),
}
summary_path = os.path.join(OUT_DIR, f"summary_{model_type}_{ts_tag}.json")
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)
print(f"[SAVED] {summary_path}")

# (Optional) Quick sanity print
print(f"[SUMMARY] test R2 mean={np.nanmean([s[0] for s in test_scores]):.4f}, "
      f"RMSE mean={np.nanmean([s[1] for s in test_scores]):.4f}, "
      f"MAE mean={np.nanmean([s[2] for s in test_scores]):.4f}")


# ===========================
# 9) (Optional) Negative Controls
# ===========================
if RUN_NEGCTRL:
    try:
        from ml_utils import negative_controls_once
        print("\n========== NEGATIVE CONTROLS ==========")
        negative_controls_once(
            X_trainval, y_trainval, pid_trainval,
            X_test, y_test,
            best_params, model_type, device
        )
    except Exception as e:
        print(f"[WARN] Negative controls skipped due to error: {e}")

print(f"\nDone. See: {OUT_DIR}")



[DBG] Normalized to (N,T,C)=(11979, 150, 77), |features|=77
[DBG] SPLIT ready (GAP=10).
[DBG] HV_MODE=none, keep_ratio=1.000
[DBG] Post-centering: train=(9626, 150, 77), val=(2196, 150, 77), test=(157, 150, 77)
🔍 Masking EDA_Tonic (1/77)
🔍 Masking EDA_Tonic_diff1 (2/77)
🔍 Masking EDA_Tonic_diff2 (3/77)
🔍 Masking EDA_Tonic_ma60 (4/77)
🔍 Masking EDA_Tonic_std150 (5/77)
🔍 Masking EDA_Tonic_slope (6/77)
🔍 Masking EDA_Tonic_iqr (7/77)
🔍 Masking EDA_Phasic (8/77)
🔍 Masking EDA_Phasic_diff1 (9/77)
🔍 Masking EDA_Phasic_diff2 (10/77)
🔍 Masking EDA_Phasic_ma60 (11/77)
🔍 Masking EDA_Phasic_std150 (12/77)
🔍 Masking EDA_Phasic_slope (13/77)
🔍 Masking EDA_Phasic_iqr (14/77)
🔍 Masking SCR_Amplitude (15/77)
🔍 Masking SCR_Amplitude_diff1 (16/77)
🔍 Masking SCR_Amplitude_diff2 (17/77)
🔍 Masking SCR_Amplitude_ma60 (18/77)
🔍 Masking SCR_Amplitude_std150 (19/77)
🔍 Masking SCR_Amplitude_slope (20/77)
🔍 Masking SCR_Amplitude_iqr (21/77)
🔍 Masking SCR_RiseTime (22/77)
🔍 Masking SCR_RiseTime_diff1 (23/77)
🔍 Mas